# Create high wage percentages across regions and industries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import warnings
from jqi_functions import *
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format='retina'

## Loading Data

In [2]:
ipums = pd.read_csv('data/IPUMS_2019.csv')

In [3]:
ipums_titles = pd.read_csv('data/ind_indnaics_crosswalk_2000_onward_without_code_descriptions.csv')
ipums_titles = ipums_titles.iloc[2:]
ipums_titles = ipums_titles.iloc[:,10:]

In [4]:
county_info = pd.read_csv('data/county_to_regions_key - Sheet1.csv')

In [56]:
cost_of_living = pd.read_csv('data/united-way-col-1A1PS1C2019.csv') # 1 adult, 1 preschooler, 1 child
cost_of_living = cost_of_living.iloc[0:14, :2]

In [6]:
naics_parsed_crosswalk = pd.read_csv('data/naics_parsed_crosswalk.csv').drop_duplicates(subset='INDNAICS').reset_index().iloc[:,1:]

## Cleaning Data

In [7]:
ipums = ipums[['STATEFIP', 
               'COUNTYFIP',
              'INDNAICS',
              'PERWT',
              'INCWAGE']]

In [8]:
ca_ipums = ipums.loc[ipums['STATEFIP'] == 6].copy()
ca_ipums = ca_ipums.reset_index()

In [9]:
ca_ipums['INDNAICS'] = normalize_titles(ca_ipums['INDNAICS'])

In [10]:
ipums_titles['2018 Onward ACS/PRCS INDNAICS CODE'] = normalize_titles(ipums_titles['2018 Onward ACS/PRCS INDNAICS CODE'])

## Merging IPUMS Data

In [11]:
ipums_merged = pd.merge(ca_ipums, ipums_titles,
                              left_on = 'INDNAICS', right_on = '2018 Onward ACS/PRCS INDNAICS CODE')

In [12]:
ipums_merged = pd.merge(ipums_merged, naics_parsed_crosswalk, on='INDNAICS')

In [13]:
show_null(ipums_merged)

TOTAL ROWS: 224526
index: 0 null values, 0.00%
STATEFIP: 0 null values, 0.00%
COUNTYFIP: 0 null values, 0.00%
INDNAICS: 0 null values, 0.00%
PERWT: 0 null values, 0.00%
INCWAGE: 0 null values, 0.00%
2018 Onward ACS/PRCS INDNAICS CODE: 0 null values, 0.00%
Industry Title: 0 null values, 0.00%
Parsed_Code: 0 null values, 0.00%


## Merging on counties

In [14]:
ipums_w_counties = pd.merge(ipums_merged, county_info, on='COUNTYFIP', how='left')
ipums_w_col = pd.merge(ipums_w_counties, cost_of_living, left_on='CDI Regions', right_on = 'Regions', how='left')

In [15]:
ipums_regions = ipums_w_col[['INDNAICS', 'Parsed_Code', 
                                          'INCWAGE',
                                          'Industry Title',
                                          'County',
                                          'CDI Regions','Rural/Urban',
                                          'PERWT', 'Cost of Living']]
ipums_regions = ipums_regions.dropna(subset=['Cost of Living'])

In [53]:
ipums_regions.head()

,INDNAICS,Parsed_Code,INCWAGE,Industry Title,County,CDI Regions,Rural/Urban,PERWT,Cost of Living,Above CA Threshold,wt_ind_counts,wt_CA_above_thresh,wt_CA_high_wage_count,wt_CA_high_wage_perc,unweighted_ind_counts
0,4853,485,23100,Taxi and limousine service,Los Angeles,Los Angeles,Urban,21.0,80216.0,0,130256.0,0.0,7350.0,5.642734,1136
1,4853,485,28000,Taxi and limousine service,Los Angeles,Los Angeles,Urban,11.0,80216.0,0,130256.0,0.0,7350.0,5.642734,1136
2,4853,485,1000,Taxi and limousine service,San Francisco,Bay Area,Urban,4.0,93392.0,0,130256.0,0.0,7350.0,5.642734,1136
3,4853,485,1000,Taxi and limousine service,San Francisco,Bay Area,Urban,147.0,93392.0,0,130256.0,0.0,7350.0,5.642734,1136
4,4853,485,28000,Taxi and limousine service,Los Angeles,Los Angeles,Urban,35.0,80216.0,0,130256.0,0.0,7350.0,5.642734,1136


## Cost of Living Threshold - State Level

### 1 adult, 1 preschooler, 1 school aged child

In [16]:
ca_cost_of_living = cost_of_living.iloc[11].values[1] # from united way real cost measure - weighted average

In [17]:
ipums_regions['Above CA Threshold'] = ipums_regions['INCWAGE'] > ca_cost_of_living
ipums_regions['wt_ind_counts'] = ipums_regions['PERWT'].groupby(ipums_regions['Industry Title']).transform('sum')
ipums_regions["Above CA Threshold"] = ipums_regions["Above CA Threshold"].astype(int)
ipums_regions["wt_CA_above_thresh"] = ipums_regions["Above CA Threshold"] * ipums_regions['PERWT']
ipums_regions['wt_CA_high_wage_count'] = ipums_regions['wt_CA_above_thresh'].groupby(
    ipums_regions['Industry Title']).transform('sum')
ipums_regions['wt_CA_high_wage_perc'] = (ipums_regions['wt_CA_high_wage_count'] / ipums_regions['wt_ind_counts']) * 100
ipums_regions['unweighted_ind_counts'] = ipums_regions['Industry Title'].groupby(ipums_regions['Industry Title']).transform('count')

In [18]:
high_wage_ca_wt = ipums_regions.drop_duplicates(subset='Industry Title')
high_wage_ca_wt = high_wage_ca_wt[['Industry Title', 'INDNAICS', 'Parsed_Code', 'unweighted_ind_counts','wt_ind_counts', 'wt_CA_high_wage_count', 'wt_CA_high_wage_perc']]
high_wage_ca_wt = high_wage_ca_wt.sort_values(by='wt_CA_high_wage_perc', ascending=False)
high_wage_ca_wt = high_wage_ca_wt.reset_index().iloc[:,1:]

In [19]:
high_wage_ca_wt.head(10)

,Industry Title,INDNAICS,Parsed_Code,unweighted_ind_counts,wt_ind_counts,wt_CA_high_wage_count,wt_CA_high_wage_perc
0,Software publishers,5112,511,403,40200.0,30290.0,75.348259
1,Internet publishing and broadcasting and web s...,51913,519,890,83933.0,62925.0,74.970512
2,Not specified utilities,22s,220,40,3206.0,2140.0,66.749844
3,Pipeline transportation,486,400,16,1672.0,1099.0,65.729665
4,Computer systems design and related services,5415,541,6168,606792.0,398057.0,65.600239
5,"Electric and gas, and other combinations",221mp,221,340,29657.0,18917.0,63.785953
6,Computer and peripheral equipment manufacturing,3341,334,516,47733.0,29545.0,61.896382
7,Aerospace products and parts manufacturing,33641m2,336,176,16206.0,9907.0,61.131680
8,"Data processing, hosting, and related services",5182,518,272,25713.0,14667.0,57.041185
9,"Electric power generation, transmission and di...",2211p,221,527,49414.0,27968.0,56.599344


## Cost of Living Threshold - Rural vs. Urban

In [75]:
df_rural = ipums_regions.loc[ipums_regions['Rural/Urban'] == 'Rural'].copy()
df_urban = ipums_regions.loc[ipums_regions['Rural/Urban'] == 'Urban'].copy()

In [76]:
rural_col = cost_of_living.iloc[13].values[1]
urban_col = cost_of_living.iloc[12].values[1]

In [77]:
def add_to_community_df(df, col):
    df['above_comm_thresh'] = df['INCWAGE'] > col
    df['wt_comm_ind_counts'] = df['PERWT'].groupby(df['Industry Title']).transform('sum')
    df["above_comm_thresh"] = df["above_comm_thresh"].astype(int)
    df["wt_comm_above_thresh"] = df["above_comm_thresh"] * df['PERWT']
    df['wt_comm_high_wage_count'] = df['wt_comm_above_thresh'].groupby(
        df['Industry Title']).transform('sum')
    df['wt_comm_high_wage_perc'] = (df['wt_comm_high_wage_count'] / df['wt_comm_ind_counts']) * 100
    return df

In [78]:
df_rural = add_to_community_df(df_rural, rural_col)
df_urban = add_to_community_df(df_urban, urban_col)

In [79]:
df_rural.head()

,INDNAICS,Parsed_Code,INCWAGE,Industry Title,County,CDI Regions,Rural/Urban,PERWT,Cost of Living,Above CA Threshold,wt_ind_counts,wt_CA_above_thresh,wt_CA_high_wage_count,wt_CA_high_wage_perc,unweighted_ind_counts,above_comm_thresh,wt_comm_ind_counts,wt_comm_above_thresh,wt_comm_high_wage_count,wt_comm_high_wage_perc
39,4853,485,33600,Taxi and limousine service,Madera,Central Valley,Rural,304.0,56747.0,0,130256.0,0.0,7350.0,5.642734,1136,0,6021.0,0.0,259.0,4.301611
46,4853,485,30000,Taxi and limousine service,Napa,Bay Area,Rural,269.0,93392.0,0,130256.0,0.0,7350.0,5.642734,1136,0,6021.0,0.0,259.0,4.301611
48,4853,485,0,Taxi and limousine service,Shasta,Shasta / Cascades,Rural,57.0,56468.0,0,130256.0,0.0,7350.0,5.642734,1136,0,6021.0,0.0,259.0,4.301611
77,4853,485,0,Taxi and limousine service,Shasta,Shasta / Cascades,Rural,107.0,56468.0,0,130256.0,0.0,7350.0,5.642734,1136,0,6021.0,0.0,259.0,4.301611
81,4853,485,0,Taxi and limousine service,Shasta,Shasta / Cascades,Rural,57.0,56468.0,0,130256.0,0.0,7350.0,5.642734,1136,0,6021.0,0.0,259.0,4.301611


In [80]:
def get_comm_high_wage(df, new_df):
    new_df = df.drop_duplicates(subset='Industry Title')
    new_df = new_df[['Rural/Urban','Industry Title', 'INDNAICS', 'Parsed_Code', 'unweighted_ind_counts',
                     'wt_comm_ind_counts', 'wt_comm_high_wage_count', 'wt_comm_high_wage_perc']]
    new_df = new_df.sort_values(by='wt_comm_high_wage_perc', ascending=False)
    new_df = new_df.reset_index().iloc[:,1:]
    return new_df

In [81]:
df_rural_high_wage = df_rural.copy()
df_urban_high_wage = df_urban.copy()

In [82]:
df_rural_high_wage = get_comm_high_wage(df_rural, df_rural.copy())
df_urban_high_wage = get_comm_high_wage(df_urban, df_urban.copy())

In [83]:
comms_high_wage_merged = pd.concat([df_rural_high_wage, df_urban_high_wage])

In [84]:
comms_high_wage_merged_ = comms_high_wage_merged.groupby(['Rural/Urban','Industry Title']).agg(lambda x: x)
comms_high_wage_merged_ = comms_high_wage_merged_.sort_values(['Rural/Urban', 'wt_comm_high_wage_perc'], ascending=(True, False))

In [86]:
comms_high_wage_merged_

INDNAICS  \
Rural/Urban Industry Title                                                
Rural       Footwear Manufacturing                                 3162   
            Household appliances                                   3352   
            Metals and minerals, except petroleum, merchant...     4235   
            Miscellaneous paper and pulp products                 3222m   
            Nonferrous metal, except aluminum, production a...     3314   
...                                                                 ...   
Urban       Book stores and news dealers                          45121   
            Bowling centers                                       71395   
            Railroad rolling stock manufacturing                   3365   
            Tobacco                                                3122   
            Unemployed, with no work experience in past 5 y...   999920   

                                                                Parsed_Code  \
Rural/Urban Industry Title                                                    
Rural       Footwear Manufacturing                                      315   
            Household appliances                                        335   
            Metals and minerals, except petroleum, merchant...          423   
            Miscellaneous paper and pulp products                       322   
            Nonferrous metal, except aluminum, production a...          331   
...                                                                     ...   
Urban       Book stores and news dealers                                451   
            Bowling centers                                             713   
            Railroad rolling stock manufacturing                        336   
            Tobacco                                                     312   
            Unemployed, with no work experience in past 5 y...          999   

                                                                unweighted_ind_counts  \
Rural/Urban Industry Title                                                              
Rural       Footwear Manufacturing                                                 28   
            Household appliances                                                   31   
            Metals and minerals, except petroleum, merchant...                     75   
            Miscellaneous paper and pulp products                                  38   
            Nonferrous metal, except aluminum, production a...                     25   
...                                                                               ...   
Urban       Book stores and news dealers                                          154   
            Bowling centers                                                        39   
            Railroad rolling stock manufacturing                                   14   
            Tobacco                                                                 1   
            Unemployed, with no work experience in past 5 y...                   1959   

                                                                wt_comm_ind_counts  \
Rural/Urban Industry Title                                                           
Rural       Footwear Manufacturing                                            66.0   
            Household appliances                                              54.0   
            Metals and minerals, except petroleum, merchant...               118.0   
            Miscellaneous paper and pulp products                             78.0   
            Nonferrous metal, except aluminum, production a...               138.0   
...                                                                            ...   
Urban       Book stores and news dealers                                   12248.0   
            Bowling centers                                                 3902.0   
            Railroad rolling stock manufa

## Cost of Living Threshold - Regional Level

In [20]:
regions = ipums_regions['CDI Regions'].unique()

In [21]:
df_LA = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[0]].copy()
df_Bay_Area = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[1]].copy()
df_Cent_Coast = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[2]].copy()
df_Cent_Valley = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[3]].copy()
df_IE = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[4]].copy()
df_OC = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[5]].copy()
df_Sac = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[6]].copy()
df_SD = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[7]].copy()
df_Shasta = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[8]].copy()
df_Redwood_Coast = ipums_regions.loc[ipums_regions['CDI Regions'] == regions[9]].copy()

In [22]:
region_dfs = [df_LA, df_Bay_Area, df_Cent_Coast, df_Cent_Valley, df_IE, df_OC,
              df_Sac, df_SD, df_Shasta, df_Redwood_Coast]

In [23]:
def add_to_region_df(df):
    df['above_region_thresh'] = df['INCWAGE'] > df['Cost of Living']
    df['wt_reg_ind_counts'] = df['PERWT'].groupby(df['Industry Title']).transform('sum')
    df["above_region_thresh"] = df["above_region_thresh"].astype(int)
    df["wt_reg_above_thresh"] = df["above_region_thresh"] * df['PERWT']
    df['wt_reg_high_wage_count'] = df['wt_reg_above_thresh'].groupby(
        df['Industry Title']).transform('sum')
    df['wt_reg_high_wage_perc'] = (df['wt_reg_high_wage_count'] / df['wt_reg_ind_counts']) * 100
    return df

In [24]:
for df in region_dfs:
    df = add_to_region_df(df)

In [25]:
def get_region_high_wage(df, new_df):
    new_df = df.drop_duplicates(subset='Industry Title')
    new_df = new_df[['CDI Regions', 'Industry Title', 'INDNAICS', 'Parsed_Code', 'Cost of Living', 'unweighted_ind_counts',
                     'wt_reg_ind_counts', 'wt_reg_high_wage_count', 'wt_reg_high_wage_perc']]
    new_df = new_df.sort_values(by='wt_reg_high_wage_perc', ascending=False)
    new_df = new_df.reset_index().iloc[:,1:]
    return new_df

In [26]:
# initialize new dfs
df_LA_high_wage = df_LA.copy()
df_Bay_Area_high_wage = df_Bay_Area.copy()
df_Cent_Coast_high_wage = df_Cent_Coast.copy()
df_Cent_Valley_high_wage = df_Cent_Valley.copy()
df_IE_high_wage = df_IE.copy()
df_OC_high_wage = df_OC.copy()
df_Sac_high_wage = df_Sac.copy()
df_SD_high_wage = df_SD.copy()
df_Shasta_high_wage = df_Shasta.copy()
df_Redwood_Coast_high_wage = df_Redwood_Coast.copy()

In [27]:
region_high_wage_dfs = [df_LA_high_wage, df_Bay_Area_high_wage, df_Cent_Coast_high_wage, 
                        df_Cent_Valley_high_wage, df_IE_high_wage, df_OC_high_wage,
              df_Sac_high_wage, df_SD_high_wage, df_Shasta_high_wage, df_Redwood_Coast_high_wage]

In [28]:
for i in range(len(region_dfs)):
    region_high_wage_dfs[i] = get_region_high_wage(region_dfs[i], region_dfs[i].copy())

In [29]:
regions_high_wage_merged = pd.concat(region_high_wage_dfs)

In [30]:
regions_high_wage_merged_ = regions_high_wage_merged.groupby(['CDI Regions','Industry Title']).agg(lambda x: x)
regions_high_wage_merged_ = regions_high_wage_merged_.sort_values(['CDI Regions', 'wt_reg_high_wage_perc'], ascending=(True, False))

In [31]:
regions_high_wage_merged_.head(10)

INDNAICS  \
CDI Regions Industry Title                                                
Bay Area    Metal ore mining                                       2122   
            Tires                                                 32621   
            Veneer, plywood, and engineered wood products          3212   
            Nonmetallic mineral mining and quarrying               2123   
            Internet publishing and broadcasting and web se...    51913   
            Software publishers                                    5112   
            Footwear Manufacturing                                 3162   
            Agricultural implements                               33311   
            Paint, coating, and adhesives                          3255   
            Computer and peripheral equipment manufacturing        3341   

                                                                Parsed_Code  \
CDI Regions Industry Title                                                    
Bay Area    Metal ore mining                                            212   
            Tires                                                       326   
            Veneer, plywood, and engineered wood products               321   
            Nonmetallic mineral mining and quarrying                    212   
            Internet publishing and broadcasting and web se...          519   
            Software publishers                                         511   
            Footwear Manufacturing                                      315   
            Agricultural implements                                     333   
            Paint, coating, and adhesives                               325   
            Computer and peripheral equipment manufacturing             334   

                                                                Cost of Living  \
CDI Regions Industry Title                                                       
Bay Area    Metal ore mining                                           93392.0   
            Tires                                                      93392.0   
            Veneer, plywood, and engineered wood products              93392.0   
            Nonmetallic mineral mining and quarrying                   93392.0   
            Internet publishing and broadcasting and web se...         93392.0   
            Software publishers                                        93392.0   
            Footwear Manufacturing                                     93392.0   
            Agricultural implements                                    93392.0   
            Paint, coating, and adhesives                              93392.0   
            Computer and peripheral equipment manufacturing            93392.0   

                                                                unweighted_ind_counts  \
CDI Regions Industry Title                                                              
Bay Area    Metal ore mining                                                       11   
            Tires                                                                  23   
            Veneer, plywood, and engineered wood products                          12   
            Nonmetallic mineral mining and quarrying                               58   
            Internet publishing and broadcasting and web se...                    890   
            Software publishers                                                   403   
            Footwear Manufacturing                                                 28   
            Agricultural implements                                                25   
            Paint, coating, and adhesives                                          59   
            Computer and peripheral equipment manufacturing                       516   

                                                                wt_reg_ind_counts  \
CDI Regions Industry Title                                                

## EDD Employment Counts

### Loading manually parsed data

In [32]:
edd_merged = pd.read_csv('data/edd_merged.csv')
ca_ipums_merged = pd.read_csv('data/ca_ipums_merged.csv')

In [33]:
ca_ipums_merged['Parsed_Code'] = ca_ipums_merged['Parsed_Code'].astype(int)

In [34]:
edd_ipums_merged = pd.merge(ca_ipums_merged, edd_merged, on='Parsed_Code')

In [35]:
thresholds = pd.read_feather('data/high_wage_ca_wt.fea')

In [36]:
thresholds_edd = pd.merge(edd_ipums_merged, thresholds, on='INDNAICS')
thresholds_edd = thresholds_edd.drop_duplicates(subset='Industry Title')

### High wage employment per industry in California

In [37]:
thresholds_edd = thresholds_edd[['Industry Title', 'INDNAICS', 'Parsed_Code', 'COUNTYFIP', 'unweighted_ind_counts', 'wt_ind_counts', 'wt_CA_high_wage_count', 'wt_CA_high_wage_perc', 'Current Employment']]
thresholds_edd['High Wage Employment'] = round((thresholds_edd['wt_CA_high_wage_perc'] / 100) * thresholds_edd['Current Employment'])
thresholds_edd = thresholds_edd.sort_values(by='wt_CA_high_wage_perc', ascending=False)
thresholds_edd = thresholds_edd.reset_index().iloc[:,1:]

In [38]:
thresholds_edd.head(10)

,Industry Title,INDNAICS,Parsed_Code,COUNTYFIP,unweighted_ind_counts,wt_ind_counts,wt_CA_high_wage_count,wt_CA_high_wage_perc,Current Employment,High Wage Employment
0,Tobacco,3122,312,0,1,55.0,55.0,100.000000,64000,64000.0
1,Coal mining,2121,212,19,3,713.0,713.0,100.000000,5900,5900.0
2,Pipeline transportation,486,400,67,16,1672.0,1672.0,100.000000,3939400,3939400.0
3,"Engine, turbine, and power transmission equipm...",3336,333,73,52,6383.0,5722.0,89.644368,137400,123171.0
4,Software publishers,5112,511,59,403,40200.0,35362.0,87.965174,251800,221496.0
5,Sewage treatment facilities,22132,221,41,100,9362.0,8090.0,86.413160,52200,45108.0
6,"Electric and gas, and other combinations",221mp,221,37,340,29657.0,25471.0,85.885288,52200,44832.0
7,Internet publishing and broadcasting and web s...,51913,519,37,890,83933.0,70992.0,84.581750,125100,105812.0
8,Not specified utilities,22s,220,37,40,3206.0,2711.0,84.560200,57200,48368.0
9,Nonmetallic mineral mining and quarrying,2123,212,99,58,4867.0,4038.0,82.966920,5900,4895.0


### High wage employment per industry per region

### Loading data by county

In [39]:
full_edd = pd.read_csv('data/Current_EDD_1121.csv')
edd_crosswalk = pd.read_excel('data/Industry_Title_Crosswalk.xlsx')
county_edd = full_edd.loc[full_edd['Area Type'] == 'County'].copy()
# county_edd = county_edd.loc[county_edd['Date'] == '12/01/2019'].copy()
# county_edd = county_edd.loc[county_edd['Seasonally Adjusted'] == 'N'].copy()
county_edd['Area Name'] = county_edd['Area Name'].str.replace(' County', '')
county_edd = county_edd.reset_index()

In [40]:
county_edd_merged = pd.merge(county_edd, edd_crosswalk, left_on='Industry Title', right_on='EDD Industry Title')

In [41]:
regions_edd = pd.merge(county_edd_merged, county_info, left_on='Area Name', right_on='County')

In [42]:
regions_edd = regions_edd[['Industry Title', 'Current Employment', 'Parsed_Code', 'COUNTYFIP', 'County', 'CDI Regions']]

In [43]:
edd_LA = regions_edd.loc[regions_edd['CDI Regions'] == regions[0]].copy()
edd_Bay_Area = regions_edd.loc[regions_edd['CDI Regions'] == regions[1]].copy()
edd_Cent_Coast = regions_edd.loc[regions_edd['CDI Regions'] == regions[2]].copy()
edd_Cent_Valley = regions_edd.loc[regions_edd['CDI Regions'] == regions[3]].copy()
edd_IE = regions_edd.loc[regions_edd['CDI Regions'] == regions[4]].copy()
edd_OC = regions_edd.loc[regions_edd['CDI Regions'] == regions[5]].copy()
edd_Sac = regions_edd.loc[regions_edd['CDI Regions'] == regions[6]].copy()
edd_SD = regions_edd.loc[regions_edd['CDI Regions'] == regions[7]].copy()
edd_Shasta = regions_edd.loc[regions_edd['CDI Regions'] == regions[8]].copy()
edd_Redwood_Coast = regions_edd.loc[regions_edd['CDI Regions'] == regions[9]].copy()

In [44]:
def get_region_edd(df):
    df['Current Employment'] = df['Current Employment'].groupby(df['Industry Title']).transform('sum')
    df = df[['CDI Regions', 'Industry Title', 'Current Employment', 'Parsed_Code']]
    df = df.drop_duplicates(subset='Industry Title')
    df = df.reset_index().iloc[:,1:]
    return df

In [45]:
edd_LA = get_region_edd(edd_LA)
edd_Bay_Area = get_region_edd(edd_Bay_Area)
edd_Cent_Coast = get_region_edd(edd_Cent_Coast)
edd_Cent_Valley = get_region_edd(edd_Cent_Valley)
edd_IE = get_region_edd(edd_IE)
edd_OC = get_region_edd(edd_OC)
edd_Sac = get_region_edd(edd_Sac)
edd_SD = get_region_edd(edd_SD)
edd_Shasta = get_region_edd(edd_Shasta)
edd_Redwood_Coast = get_region_edd(edd_Redwood_Coast)

In [46]:
high_wage_LA = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[0]].copy()
high_wage_Bay_Area = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[1]].copy()
high_wage_Cent_Coast = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[2]].copy()
high_wage_Cent_Valley = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[3]].copy()
high_wage_IE = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[4]].copy()
high_wage_OC = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[5]].copy()
high_wage_Sac = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[6]].copy()
high_wage_SD = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[7]].copy()
high_wage_Shasta = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[8]].copy()
high_wage_Redwood_Coast = regions_high_wage_merged.loc[regions_high_wage_merged['CDI Regions'] == regions[9]].copy()

In [47]:
def get_region_high_wage(df, edd_df):
    df['parsed_ind_counts'] = df['Parsed_Code'].groupby(df['Parsed_Code']).transform('count')
    df['parsed_perc_sum'] = df['wt_reg_high_wage_perc'].groupby(df['Parsed_Code']).transform('sum')
    df['final_high_wage_perc'] = df['parsed_perc_sum'] / df['parsed_ind_counts']
    df = df.sort_values(by='final_high_wage_perc', ascending=False)
    df = pd.merge(df, edd_df, on='Parsed_Code')
    df['High Wage Count'] = round(df['Current Employment'] * (df['final_high_wage_perc'] / 100))
    df = df[['CDI Regions_x', 'Industry Title_x', 'Parsed_Code', 'INDNAICS', 'Cost of Living', 'unweighted_ind_counts', 'wt_reg_ind_counts', 'wt_reg_high_wage_perc', 'final_high_wage_perc', 'Current Employment', 'High Wage Count']].reset_index().iloc[:,1:]
    return df

In [48]:
high_wage_LA = get_region_high_wage(high_wage_LA, edd_LA)
high_wage_Bay_Area = get_region_high_wage(high_wage_Bay_Area, edd_Bay_Area)
high_wage_Cent_Coast = get_region_high_wage(high_wage_Cent_Coast, edd_Cent_Coast)
high_wage_Cent_Valley = get_region_high_wage(high_wage_Cent_Valley, edd_Cent_Valley)
high_wage_IE = get_region_high_wage(high_wage_IE, edd_IE)
high_wage_OC = get_region_high_wage(high_wage_OC, edd_OC)
high_wage_Sac = get_region_high_wage(high_wage_Sac, edd_Sac)
high_wage_SD = get_region_high_wage(high_wage_SD, edd_SD)
high_wage_Shasta = get_region_high_wage(high_wage_Shasta, edd_Shasta)
high_wage_Redwood_Coast = get_region_high_wage(high_wage_Redwood_Coast, edd_Redwood_Coast)

In [49]:
region_high_wage_dfs = [high_wage_LA, high_wage_Bay_Area, high_wage_Cent_Coast, 
                        high_wage_Cent_Valley, high_wage_IE, high_wage_OC,
              high_wage_Sac, high_wage_SD, high_wage_Shasta, high_wage_Redwood_Coast]

In [50]:
regions_high_wage_concat = pd.concat(region_high_wage_dfs)

In [51]:
regions_high_wage_concat = regions_high_wage_concat.sort_values(['CDI Regions_x', 'final_high_wage_perc'], ascending=(True, False))

In [52]:
regions_high_wage_concat.head(10)

,CDI Regions_x,Industry Title_x,Parsed_Code,INDNAICS,Cost of Living,unweighted_ind_counts,wt_reg_ind_counts,wt_reg_high_wage_perc,final_high_wage_perc,Current Employment,High Wage Count
0,Bay Area,Management of companies and enterprises,550,55,93392.0,320,8825.0,61.371105,61.371105,21393900,13129673.0
1,Bay Area,"Justice, public order, and safety activities",922,92mp,93392.0,3485,55406.0,41.558676,41.558676,2987100,1241399.0
2,Bay Area,Public finance activities,921,92113,93392.0,454,6063.0,43.328385,37.375334,748800,279867.0
3,Bay Area,Other general government and support,921,92119,93392.0,248,4027.0,40.501614,37.375334,748800,279867.0
4,Bay Area,Executive offices and legislative bodies,921,9211mp,93392.0,1764,38797.0,28.296002,37.375334,748800,279867.0
5,Bay Area,Administration of economic programs and space ...,920,92m2,93392.0,657,14284.0,28.395407,33.086363,147130000,48679965.0
6,Bay Area,Administration of economic programs and space ...,920,92m2,93392.0,657,14284.0,28.395407,33.086363,30311700,10029039.0
7,Bay Area,Administration of environmental quality and ho...,920,92m1,93392.0,392,6915.0,48.358641,33.086363,147130000,48679965.0
8,Bay Area,Administration of environmental quality and ho...,920,92m1,93392.0,392,6915.0,48.358641,33.086363,30311700,10029039.0
9,Bay Area,Administration of human resource programs,920,923,93392.0,1939,37205.0,22.505040,33.086363,147130000,48679965.0
